* RNN  
![screensh](https://t1.daumcdn.net/cfile/tistory/2578204757AC186F2A)  


* RNN을 공식으로 표현  
![screensh](https://t1.daumcdn.net/cfile/tistory/271F9B4757AC187002)      
 
* Vanilla RNN  
![screensh](https://t1.daumcdn.net/cfile/tistory/21140B4757AC18710F)   


* 같은 함수에 대해 같은 입력이 똑같이 사용  
![screensh](https://t1.daumcdn.net/cfile/tistory/274D224F57AC187225)  
 
* 4가지 종류의 글자가 있기 때문에 크기가 4인 리스트로 처리   
![screensh](https://t1.daumcdn.net/cfile/tistory/2648FF4F57AC18732A)  
 
 
* 최종적으로는 모든 단계에서 값을 예측하고 실제 값과 비교 가능  
![screensh](https://t1.daumcdn.net/cfile/tistory/2455824F57AC18731A)  

* 바닐라 RNN을 변형해서 만들 수 있는 모델들   
![screensh](https://t1.daumcdn.net/cfile/tistory/236C7D4F57AC187405)  
  
  
one-to-one : 가장 단순한 바닐라 RNN  
ont-to-many : 이미지에 대한 설명을 붙일 때 사용 ex) "소년이 사과를 고르고 있다"  
many-to-one : 감정상태를 예측할 때 사용   
many-to-many :    
① 기계 번역에서 사용 여러 개의 단어로 구성된 문장을 입력으로 받아서  
여러 개의 단어로 구성된 문장을 반환한다.   
② 동영상 같은 경우는 여러 개의 이미지 프레임에 대해 여러 개의 설명이나 변역 형태로
결과를 반환하게 된다.  

* RNN도 여러 개의 layer를 두고 복잡한 형태로 구성 가능  

![screensh](https://t1.daumcdn.net/cfile/tistory/2249F34E57AC187736)   


In [ ]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  
  입력(inputs) 및 타겟(targets) 데이터는 int형 list이다.
  hprev는 초기 hidden state의 Hx1 배열.
  loss와 gradient, hidden state를 return 한다.
  
  dWxh, dWhh, dWhy, dbh, dby -> gradient인듯.
  
  https://pythonkim.tistory.com/57 -> 그림으로 설명 잘 되어 있음
  
  """
    xs, hs, ys, ps = {}, {}, {}, {} # hs -> hidden state인듯.
    hs[-1] = np.copy(hprev) 
    # hprev를 hidden state에 copy.
    # 밑에 for문을 돌렸을 때, hs[t]를 만드려면 hs[t-1]이 필요한데,
    # for문이 0부터 돌아서 hs[0]을 구하기 위해 hs[-1]에 저장
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # 1 of k 부호화 -> 해당되는 클래스만 1로 표시 나머지 0
    xs[t][inputs[t]] = 1 
    # depth==2인 xs[t]
    # xs[0]에는 input[0], xs[1]에는 inputs[1] .. xs[t]에는 inputs[n]을 저장.
    
    
    
    # ↓↓↓↓↓ 여기 for문으로 수정하면 hidden state를 더 늘릴 수 있을 듯!
    
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)  # hidden state 
    # tanh() 함수는 기존의 sigmoid 함수를 개량한 함수.
    # sigmoid는 0~1값을 반환하는데, tanh는 -1~1사이의 값이 반환된다.
    # 그래서 hidden layer에 있는 값들도 해당 범위에 존재하게 된다.
    # h_t = f_w(h_t-1,x_t) -> rnn 공식 
    # f_w는 some function, parameters == w
    # hs[t] == new state
    # hs[t-1] == old state
    # xs[t] == input vector
    # ==> hs[t]를 바로 직전의 state인 hs[t-1]과
    # 현재 input인 xs[t]를 이용하여 update 한다고 보면 될 듯.
    
    # ↓↓↓↓↓ 그림으로 표현 ↓↓↓↓↓
    
    # h_0   h_1   h_2   h_3   ...   h_t 
    #  A  -> A  -> A  -> A    ... -> A    
    # x_0   x_1   x_2   x_3   ...   x_t      
    
    ys[t] = np.dot(Why, hs[t]) + by # 다음 문자에 대한 정규화되지 않은 로그 확률
    
    # RNN은 step을 거칠 때마다 어떤 결과를 예측하게 된다.
    # 그리고 이런 예측 값을 y라고 한다.
    # 위에 주석으로 그린 그림에서 한 단계마다 y가 나온다고 보면 된다.
    # 여러 hidden state들을 통합적으로 하나의 rnn이라고 보면,
    # 특정 시점 t에서의 예측 값인 y는 ys[t]이다.
    
    # xs[t] -> RNN -> ys[t]
    # RNN은 hs[t]로 이루어져 있다고 생각. -> hs[0]~hs[t]까지 조합.
    
    
    # 여기까지 과정을 살펴보면, hs[t]를 먼저, hs[t-1]과 xs[t]를 이용해서 업데이트를 해주고,
    # 업데이트된 hs[t]를 이용해서 ys[t]를 구한다.
    
    # ※ 주의할점 ※
    # 같은 함수에 대해 같은 입력이 매번 똑같이 사용된다!
    # step이 다ㅡ러다라도, 같은 함수에 대해서는 같은 입력이 사용된다.
    
    # h_0   h_1   h_2   h_3   ...   h_t 
    #  A  -> A  -> A  -> A    ... -> A    
    # x_0   x_1   x_2   x_3   ...   x_t    
    
    # 이 그림으로 치면, h_0에는 x_0만이 들어간다.
    
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # 다음 문자 확률
    
    loss += -np.log(ps[t][targets[t],0]) 
    # softmax (cross-entropy loss)
    # softmax로 분류를 해서, loss를 구한다.  ->   파이썬으로 배우는 머신러닝의 교과서 Chapter 6 부분
    # 최대가능도법으로 확률이라서 -를 붙여준다. (책에 나왔던 부분) 
    # -> 회귀 문제에서의 loss와 똑같이 loss가 작을수록 좋게 판단하기 위함  
      
  # backward pass: compute gradients going backwards
  # backpropagation? 
  # forward를 마친 뒤, backward 실행
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why) # zeros_like 뭔지 모르겠음
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [ ]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  
  모델에서 int의 순서로 sample을 만든다.
  h는 memory state고, seed_ix는 첫번째 단계의 seed 문자이다.
  
  그냥 샘플링하는 함수인듯
  """
  x = np.zeros((vocab_size, 1)) # 1-of-k 부호화
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('c:/app/test.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print("data has",data_size,"characters,", vocab_size," unique")
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons => hidden layer 사이즈
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias

by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print("----\n", txt,"\n ----\n")

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print("iter", n, "loss: ", smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 